<a href="https://colab.research.google.com/github/keymemory/Machine_Learning_Second/blob/main/ML2_day1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
# 파일을 4바이트 단위로 읽어서 행렬로 저장하는 함수
def _read32(bytestream):
  dt = np.dtype(np.uint32).newbyteorder('>')
  return np.frombuffer(bytestream.read(4), dtype=dy)[0]

# 테스트 셋을 저장할 변수
test_images = 0
test_labels = 0

In [ ]:
# 트레이닝 셋(트레이닝 + 벨리데이션)을 저장할 변수
train_data = 0
train_datavalue = 0

# 트레이닝 셋의 갯수
image_num = 55000

# 트레이닝 반복 횟수
training_epochs = 25

# Online Learning을 위해 나눠서 수행할 batch 크기(55000개의 데이터이므로 한번에 550개씩
# 학습 수행)
batch_size = 100

In [ ]:
# 테스트 셋(이미지데이터) 파일 읽은 후 행렬로 변환
with open('t10k-images.idx3-ubyte','rb') as f:
 magic = _read32(f) #고유한 코드(?)
 num_images = _read32(f) #데이터 번호 (1~60000)
 rows = _read32(f) #rows데이터의 구성 수 (28)
 cols = _read32(f) #cols데이터의 구성 수 (28)
 buf = f.read(rows * cols * num_images) #(28 * 28 * 데이터번호) 단위로 읽음
 
 # frombuffer를 통해 1차원 배열 형태로 8비트 형태로 변환한다.
 test_images = np.frombuffer(buf, dtype=np.uint8)

 # reshape를 통해 행렬 변환
 test_images = test_images.reshape(num_images, rows, cols, 1)

 # 위에서 생성된 행렬을 가로 행렬로 풀기 위한 수식
 test_images = test_images.reshape(test_images.shape[0],
 test_images.shape[1]*test_images.shape[2]) 

In [ ]:
 # 데이터 형식을 float32로 변환
 test_images = test_images.astype(np.float32) 

In [ ]:
# 트레이닝 셋(이미지 데이터) 파일 읽은 후 행렬로 변환.
with open('train-images.idx3-ubyte','rb') as f:
 magic = _read32(f)
 num_images = _read32(f)
 rows = _read32(f)
 cols = _read32(f)
 buf = f.read(rows * cols * num_images)
 train_data = np.frombuffer(buf, dtype=np.uint8)
 train_data = train_data.reshape(num_images, rows, cols, 1)
 train_data = train_data.reshape(train_data.shape[0],
 train_data.shape[1]*train_data.shape[2])
 train_data = train_data.astype(np.float32)

In [ ]:
# 테스트 셋(라벨 데이터) 파일 읽은 후 행렬로 변환.
with open('t10k-labels.idx1-ubyte', 'rb') as f:
 magic = _read32(f) #고유한 코드(?)
 num_items = _read32(f)
 buf = f.read(num_items)
 # frombuffer를 통해 1차원 배열 형태로 8비트 형태로 변환한다.
 _label = np.frombuffer(buf, dtype=np.uint8)
 # one_hot encording 하는 방법
 num_labels = _label.shape[0]
 index_offset = np.arange(num_labels) * 10
 test_labels = np.zeros((num_labels, 10))
 test_labels.flat[index_offset + _label.ravel()] = 1

In [ ]:
# 트레이닝 셋(라벨 데이터) 파일 읽은 후 행렬로 변환.
with open('train-labels.idx1-ubyte', 'rb') as f:
 magic = _read32(f)
 num_items = _read32(f)
 buf = f.read(num_items)
 _label = np.frombuffer(buf, dtype=np.uint8)
 num_labels = _label.shape[0]
 index_offset = np.arange(num_labels) * 10
 train_datavalue = np.zeros((num_labels, 10))
 train_datavalue.flat[index_offset + _label.ravel()] = 1

In [ ]:
# 트레이닝 데이터
x = tf.placeholder(tf.float32, [None, 784])

# 라벨 데이터
y = tf.placeholder(tf.float32, [None, 10])

# 웨이트
W = tf.Variable(tf.zeros([784, 10]))

In [ ]:
# 베리어블
b = tf.Variable(tf.zeros([10]))

# 소프트맥스 정의
activation = tf.nn.softmax(tf.matmul(x, W) + b)

# 크로스 엔트로피(코스트함수) 정의
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y * tf.log(activation),
 reduction_indices=1))

In [ ]:
# 경사 하강법 정의
optimizer = tf.train.GradientDescentOptimizer(0.000005).minimize(cross_entropy)

# 변수 초기화
init = tf.global_variables_initializer() 

In [ ]:
# 6만개의 트레이닝 셋을 55000, 5000개로 잘라서 트레이닝 셋, 벨리데이션 셋으로 나눔)
validation_images = train_data[:5000]
validation_labels = train_datavalue[:5000]
train_image = train_data[5000:]
train_labels = train_datavalue[5000:]

# 코스트의 평균값을 저장
avg_cost = 0

In [ ]:
# 학습 시작
with tf.Session() as sess:
 sess.run(init)

 #25번 반복함
 for epoch in range(training_epochs):
 start_index = 0
 end_index = 0
 # 550개 단위로 잘라서 학습을 진행함

 for step in xrange(batch_size):
 start_index = end_index;
 end_index = end_index + (image_num/batch_size)
 batch_xs = train_image[start_index:end_index]
 batch_ys = train_labels[start_index:end_index]
 sess.run(optimizer, feed_dict={x:batch_xs, y:batch_ys})
 avg_cost = sess.run(cross_entropy, feed_dict={x:batch_xs, y:batch_ys}) 

 # 학습을 한번 끝낼 때마다 디버깅 코드 출력
 print "Epoch:", '%04d' %(epoch), "cost=", "{:.9f}".format(avg_cost)
 print 'study success'

In [ ]:
 # 테스트 셋을 이용하여 성능 평가
 # 테스트 셋을 이용한 결과 값과 실제 label값을 비교연산
 correct_prediction = tf.equal(tf.argmax(activation, 1), tf.argmax(y, 1))
 
 # 결과 값의 평균을 구함
 accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
 print ("적중률:", accuracy.eval({x: test_images, y: test_labels}))